In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator, load_img

Using TensorFlow backend.


In [3]:
train_dir = '../data/rgb/train'
validation_dir = '../data/rgb/val'
image_size = 224

In [4]:
from keras.applications import VGG16

#Load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

# Freeze all the layers
for layer in vgg_conv.layers[:]:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)


from keras import models
from keras import layers
from keras import optimizers

# Create the model
model = models.Sequential()

# Add the vgg convolutional base model
model.add(vgg_conv)

# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))

# Show a summary of the model. Check the number of trainable parameters
model.summary()

<keras.engine.input_layer.InputLayer object at 0x7f19926a9eb8> False
<keras.layers.convolutional.Conv2D object at 0x7f19926b10f0> False
<keras.layers.convolutional.Conv2D object at 0x7f19926b10b8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f19926d6f60> False
<keras.layers.convolutional.Conv2D object at 0x7f19926b1588> False
<keras.layers.convolutional.Conv2D object at 0x7f1992688080> False
<keras.layers.pooling.MaxPooling2D object at 0x7f1992637400> False
<keras.layers.convolutional.Conv2D object at 0x7f1992621278> False
<keras.layers.convolutional.Conv2D object at 0x7f19925e3898> False
<keras.layers.convolutional.Conv2D object at 0x7f19926113c8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f19925aba20> False
<keras.layers.convolutional.Conv2D object at 0x7f19925ba5c0> False
<keras.layers.convolutional.Conv2D object at 0x7f199255b940> False
<keras.layers.convolutional.Conv2D object at 0x7f199258a320> False
<keras.layers.pooling.MaxPooling2D object at 0x7f199253b5c0>

In [ ]:
# No Data augmentation 
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Change the batchsize according to your system RAM
train_batchsize = 100
val_batchsize = 10

# Data Generator for Training data
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=train_batchsize,
        class_mode='categorical')

# Data Generator for Validation data
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_size, image_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

# Train the Model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=20,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)

# Save the Model
model.save('all_freezed.h5')

# Plot the accuracy and loss curves
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()